In [43]:
'''
This notebook combines the fangraphs advanced, value, and standard pages, and then merges this (inner join) on
the ESPN free agent data. We then go through many of the columns and massage/clean our data
'''

'\nThis notebook combines the fangraphs advanced, value, and standard pages, and then merges this (inner join) on\nthe ESPN free agent data. We then go through many of the columns and massage/clean our data\n'

In [44]:
import pickle
import pandas as pd
import numpy as np
import copy
import seaborn as sns

In [45]:
advanced_df = pickle.load( open( "df_advanced.pkl", "rb" ) )
standard_df = pickle.load( open( "df_standard.pkl", "rb" ) )
value_df = pickle.load( open( "df_value.pkl", "rb" ) )
free_agents_df = pickle.load( open( "free_agents_master.pkl", "rb" ) )

In [46]:
advanced_df.head()
advanced_df.count()

PA       3567
BB%      3567
K%       3567
BB/K     3567
AVG      3567
OBP      3567
SLG      3567
OPS      3567
ISO      3567
Spd      3567
BABIP    3567
UBR      3567
wGDP     3567
wSB      3567
wRC      3567
wRAA     3567
wOBA     3567
wRC+     3567
dtype: int64

In [47]:
standard_df.head()
standard_df.count()

G      3567
AB     3567
PA     3567
H      3567
1B     3567
2B     3567
3B     3567
HR     3567
R      3567
RBI    3567
BB     3567
IBB    3567
SO     3567
HBP    3567
SF     3567
SH     3567
GDP    3567
SB     3567
CS     3567
AVG    3567
dtype: int64

In [48]:
value_df.count()

Batting         3567
Base Running    3567
Fielding        3567
Positional      3567
Offense         3567
Defense         3567
League          3567
Replacement     3567
RAR             3567
WAR             3567
Dollars         3567
dtype: int64

In [49]:
#Joining on the index for both columns
#master_df becomes our primary dataframe
master_df = pd.merge(advanced_df, standard_df, how='inner',left_index=True,right_index=True)
master_df = pd.merge(master_df,value_df,how='inner',left_index=True,right_index=True)

In [50]:
master_df.count().head() #3567 - our merge worked

PA_x     3567
BB%      3567
K%       3567
BB/K     3567
AVG_x    3567
dtype: int64

In [53]:
# master_df.describe() #534 names - that sounds about right
master_df.columns
master_df.sample(2)

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars
Albert Pujols 2011,651,9.4 %,8.9 %,1.05,.299,.366,.541,.906,.242,3.9,...,-6.4,-0.8,-10.7,28.8,-11.6,1.3,18.9,37.5,4.0,$30.1
Yangervis Solarte 2015,571,6.0 %,9.8 %,0.61,.270,.320,.428,.748,.158,3.4,...,-4.7,-1.5,-0.5,1.2,-2.0,1.1,16.7,16.9,1.8,$14.4


In [54]:
#PLAYER_YEAR can be our index - easy to see what the row represents
master_df.reset_index(drop=False,inplace=True) 

In [56]:
master_df = master_df.rename({"index":"PLAYER_YEAR"},axis=1)

In [58]:
master_df.sample(2)

,PLAYER_YEAR,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,...,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars
1925,Trevor Plouffe 2013,522,6.5 %,21.5 %,0.30,.254,.309,.392,.701,.138,...,-2.5,-2.8,0.8,-7.1,-2.0,1.9,14.9,7.7,0.8,$6.1
725,Dustin Pedroia 2015,425,8.9 %,12.0 %,0.75,.291,.356,.441,.797,.150,...,-0.9,-0.7,1.3,7.3,0.6,1.4,12.4,21.7,2.3,$18.5


In [59]:
#Save progress
with open('df_MLB_Full.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)

## Data Cleaning Time

In [60]:
#BB%, K% have percentage signs
master_df[["BB%","K%"]].head()

,BB%,K%
0,13.8 %,14.4 %
1,7.5 %,16.6 %
2,13.3 %,15.2 %
3,8.0 %,15.5 %
4,19.8 %,17.9 %


In [20]:
# master_df[['Batting',
#        'Base Running', 'Fielding', 'Positional', 'Offense', 'Defense',
#        'League', 'Replacement', 'RAR', 'WAR', 'Dollars', ]]

In [61]:
#Replace % with empty
for col in ["BB%","K%"]:
    master_df[col] = master_df[col].str.replace(" \%","")

In [62]:
#Replace $ or Commas in Dollars with blank
master_df["Dollars"] = master_df["Dollars"].str.replace("\$|,","")

In [63]:
master_df.sample()

,PLAYER_YEAR,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,...,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars
748,Adam Dunn 2012,649,16.2,34.2,0.47,.204,.333,.468,.800,.263,...,-1.3,1.7,-14.1,10.5,-12.4,2.2,19.2,19.4,2.0,13.2


In [64]:
'''
Now convert all numbers available to numeric
'''
master_df = master_df.apply(pd.to_numeric,errors='ignore')
master_df.dtypes

PLAYER_YEAR      object
PA_x              int64
BB%             float64
K%              float64
BB/K            float64
AVG_x           float64
OBP             float64
SLG             float64
OPS             float64
ISO             float64
Spd             float64
BABIP           float64
UBR             float64
wGDP            float64
wSB             float64
wRC               int64
wRAA            float64
wOBA            float64
wRC+              int64
G                 int64
AB                int64
PA_y              int64
H                 int64
1B                int64
2B                int64
3B                int64
HR                int64
R                 int64
RBI               int64
BB                int64
IBB               int64
SO                int64
HBP               int64
SF                int64
SH                int64
GDP               int64
SB                int64
CS                int64
AVG_y           float64
Batting         float64
Base Running    float64
Fielding        

In [65]:
#Replace $ or Commas in DOLLARS with blank
master_df["Dollars"] = master_df["Dollars"].str.replace("(","-")
master_df["Dollars"] = master_df["Dollars"].str.replace(")","")
master_df = master_df.apply(pd.to_numeric,errors='ignore')

In [66]:
master_df["Fielding"] = master_df["Fielding"].apply(pd.to_numeric,errors='force')

In [67]:
master_df.dtypes

PLAYER_YEAR      object
PA_x              int64
BB%             float64
K%              float64
BB/K            float64
AVG_x           float64
OBP             float64
SLG             float64
OPS             float64
ISO             float64
Spd             float64
BABIP           float64
UBR             float64
wGDP            float64
wSB             float64
wRC               int64
wRAA            float64
wOBA            float64
wRC+              int64
G                 int64
AB                int64
PA_y              int64
H                 int64
1B                int64
2B                int64
3B                int64
HR                int64
R                 int64
RBI               int64
BB                int64
IBB               int64
SO                int64
HBP               int64
SF                int64
SH                int64
GDP               int64
SB                int64
CS                int64
AVG_y           float64
Batting         float64
Base Running    float64
Fielding        

In [34]:
master_df.rename({"Dollars":"Fangraphs_Dollars"}, axis='columns',inplace=True)

In [35]:
master_df[["BB%","K%"]].head()

,BB%,K%
0,13.8,14.4
1,7.5,16.6
2,13.3,15.2
3,8.0,15.5
4,19.8,17.9


In [70]:
master_df.dtypes #Looks Good

PLAYER_YEAR      object
PA_x              int64
BB%             float64
K%              float64
BB/K            float64
AVG_x           float64
OBP             float64
SLG             float64
OPS             float64
ISO             float64
Spd             float64
BABIP           float64
UBR             float64
wGDP            float64
wSB             float64
wRC               int64
wRAA            float64
wOBA            float64
wRC+              int64
G                 int64
AB                int64
PA_y              int64
H                 int64
1B                int64
2B                int64
3B                int64
HR                int64
R                 int64
RBI               int64
BB                int64
IBB               int64
SO                int64
HBP               int64
SF                int64
SH                int64
GDP               int64
SB                int64
CS                int64
AVG_y           float64
Batting         float64
Base Running    float64
Fielding        

In [72]:
#Drop AVG_y Column since AVG_x is the same number
master_df=master_df.drop(["AVG_y"],1)

In [73]:
master_df["PA_x"] = master_df["PA_x"].apply(pd.to_numeric,errors='ignore')

In [74]:
master_df.dtypes

PLAYER_YEAR      object
PA_x              int64
BB%             float64
K%              float64
BB/K            float64
AVG_x           float64
OBP             float64
SLG             float64
OPS             float64
ISO             float64
Spd             float64
BABIP           float64
UBR             float64
wGDP            float64
wSB             float64
wRC               int64
wRAA            float64
wOBA            float64
wRC+              int64
G                 int64
AB                int64
PA_y              int64
H                 int64
1B                int64
2B                int64
3B                int64
HR                int64
R                 int64
RBI               int64
BB                int64
IBB               int64
SO                int64
HBP               int64
SF                int64
SH                int64
GDP               int64
SB                int64
CS                int64
Batting         float64
Base Running    float64
Fielding        float64
Positional      

In [75]:
#Save progress
with open('df_MLB_Full.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)